<a href="https://colab.research.google.com/github/assafrbi/mini-project-Digital-Humanities/blob/main/dh_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
!pip install wikipedia
!pip install cinemagoer
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse
import concurrent.futures
import pandas as pd
import wikipedia
import numpy as np
from imdb import Cinemagoer
from tqdm import tqdm

# Maximum number of threads that will be spawned
MAX_THREADS = 50

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Load Data from IMDb

In [28]:
movie_title_arr = []
movie_year_arr = []
movie_genre_arr = []
image_id_arr = []
movie_synopsis_arr = []
need_wiki_id = []
need_wiki_name = []
need_wiki_year = []

In [29]:
ia = Cinemagoer()

def getSynopsisFromImdb(id):
  return ia.get_movie(id)['plot'][0]

In [31]:
def getMovieTitle(header):
    try:
        return header[0].find("a").getText()
    except:
        return 'NA'

def getReleaseYear(header):
    try:
        return header[0].find("span",  {"class": "lister-item-year text-muted unbold"}).getText().strip()[-5:-1]
    except:
        return 'NA'

def getGenre(muted_text):
    try:
        return muted_text.find("span",  {"class":  "genre"}).getText().strip()
    except:
        return 'NA'
        
def getsynopsys(movie, id, name, year):
    try:
        synopsis = movie.find_all("p", {"class":  "text-muted"})[1].getText().strip()
        if 'Add a Plot' in synopsis: # no imdb data -> need wiki
          need_wiki_id.append(id)
          need_wiki_name.append(name)
          need_wiki_year.append(year)
          return 'NA'

        elif 'See full summary' in synopsis:
          return getSynopsisFromImdb(id)

        return synopsis
    except:
      # no imdb data -> need wiki
      need_wiki_id.append(id)
      need_wiki_name.append(name)
      need_wiki_year.append(year)
      return 'NA'

def getImageId(image):
    try:
        return image.get('data-tconst')[2:]
    except:
        return 'NA'

In [32]:
def main(imdb_url):
    response = requests.get(imdb_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Movie Name
    movies_list  = soup.find_all("div", {"class": "lister-item mode-advanced"})
    
    for indx, movie in enumerate(movies_list):
        header = movie.find_all("h3", {"class":  "lister-item-header"})
        muted_text = movie.find_all("p", {"class":  "text-muted"})[0]
        imageDiv =  movie.find("div", {"class": "lister-item-image float-left"})
        image = imageDiv.find("img", "loadlate")
        
        # Movie Title
        movie_title = getMovieTitle(header)
        movie_title_arr.append(movie_title)

        # Movie id
        id = getImageId(image)
        image_id_arr.append(id)
        
        # Movie release year
        year = getReleaseYear(header)
        movie_year_arr.append(year)

        # Movie Synopsys
        synopsis = getsynopsys(movie, id, movie_title, year)
        movie_synopsis_arr.append(synopsis)
        
        # Genre
        genre = getGenre(muted_text)
        movie_genre_arr.append(genre)
        

In [33]:
# An array to store all the URL that are being queried
imageArr = []

# Maximum number of pages one wants to iterate over
MAX_PAGE =51

# Loop to generate all the URLS.
for i in range(1, 1500, 50):
    imdb_url = f'https://www.imdb.com/search/title/?title_type=feature&countries=il&start={i}&ref_=adv_nxt'
    imageArr.append(imdb_url)

In [34]:
def download_stories(story_urls):
    threads = min(MAX_THREADS, len(story_urls))
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(main, story_urls)

In [ ]:
# Call the download function with the array of URLS called imageArr
download_stories(imageArr)

# Attach all the data to the pandas dataframe. You can optionally write it to a CSV file as well
movieDf = pd.DataFrame({
    "Title": movie_title_arr,
    "Year": movie_year_arr,
    "Genre": movie_genre_arr,
    "Synopsis": movie_synopsis_arr,
    "IMDb id": image_id_arr,
})

needWiki = pd.DataFrame({
    "Title": need_wiki_name,
    "Year": need_wiki_year,
    "IMDb id": need_wiki_id,
})

movieDf.to_csv('israel_originated_movies_imdb.csv')
needWiki.to_csv('movies_need_wiki.csv')

In [37]:
movieDf

,Title,Year,Genre,Synopsis,IMDb id
0,The Lost Daughter,2021,Drama,A woman's beach vacation takes a dark turn whe...,9100054
1,Knock Knock,2015,"Drama, Thriller",A devoted father helps two stranded young wome...,3605418
2,Cobra,1986,"Action, Crime, Thriller",A tough-on-crime street cop must protect the o...,0090859
3,The Delta Force,1986,"Action, Adventure, Drama","After a plane is hijacked by terrorists, The D...",0090927
4,The Kindergarten Teacher,2018,"Drama, Thriller",A kindergarten teacher in New York becomes obs...,6952960
...,...,...,...,...,...
1466,Seven Days,2013,Thriller,"Eddie, an unemployed actor gets in troubles wi...",3145552
1467,A Place in Heaven,2013,Drama,NA,2960994
1468,Eddie King,1992,Drama,NA,0104164
1469,The Spy Who Never Was,1976,Drama,NA,0070812


In [38]:
needWiki

,Title,Year,IMDb id
0,Children of the Steps,1984,0147732
1,Wanted: Four Legged Husband,1993,0107542
2,The Immigrant,1991,2831500
3,Burning Land,1984,0133338
4,Ze Lo Seret Studentim,1997,0167507
...,...,...,...
489,The Loners,2009,1455618
490,Dissolution,2010,1668199
491,Seven Days,2013,3145552
492,A Place in Heaven,2013,2960994


### Load Synopses from Wikipedia - Need some tuning !!!!!!

In [40]:
wikiMovies = pd.read_csv('movies_need_wiki.csv')

In [ ]:
# movie_plot_arr = []
# movie_summary_arr = []
# wikipedia_title_arr = []

# def find_title_on_wikipedia(title, year):
#     search_results = wikipedia.search(title, results=20)
#     search_results_exact = [s for s in search_results if title in s] # results that have the exact title as part of them
    
#     if len(search_results_exact) != 0: # if the title is part of at least one result

#         # if film exists in title but not together with series or score (an example was Resident Evil 2002)
#         search_results_film = [s for s in search_results_exact if 'film' in s and 'score' not in s and 'series' not in s]

#         # Go on by additionaly searching for the year in the title
#         if len(search_results_film) == 0: # if there is no results with 'film' in the title
#             search_results_year = [s for s in search_results_exact if str(year) in s]
#             if len(search_results_year) == 0: # if there is no results with year in the title
#                 search_results_final = search_results_exact
#             else:
#                 search_results_final = search_results_year
#         else:
#             search_results_year = [s for s in search_results_film if str(year) in s]
#             if len(search_results_year) == 0:
#                 search_results_final = search_results_film
#             else:
#                 search_results_final = search_results_year  

#         # for the possible remains of the search results take the exact match or the first result
#         for s in search_results_final:
#             if s == title:
#                 title = s
#             else:
#                 title = search_results_final[0]
    
#     else: # take the first result
#         title = search_results[0]

#     return title

# def add_wikipedia():
#   section_names = ['Plot','Synopsis','Plot synopsis','Plot summary', 'Story','Plotline','The Beginning','Summary', 'Content','Premise']
#   for i in range (len(image_id_arr)):
#     imdb_title = movie_title_arr[i]
#     year = movie_year_arr[i]
#     title, page, plot, summary = np.nan, np.nan, np.nan, np.nan

#     try:
#       title = find_title_on_wikipedia(imdb_title, year)
#       search_results = wikipedia.search(title, results=20)
#       page = wikipedia.WikipediaPage(title = title)
#     except:
#         print(f'Title "{imdb_title}" does not exist on Wikipedia.')

#     for section in section_names:
#         try:
#             plot = page.section(section)
#         except:
#             print(f'Plot for -{imdb_title}- was not found.')

#     # for some movies with general title - modify the search by adding (year film) next to title
#     if plot is None:
#         imdb_title = f'{imdb_title} ({year} film)'
#         try:          
#             title = find_title_on_wikipedia(imdb_title, year)
#             search_results = wikipedia.search(title, results=20)
#             page = wikipedia.WikipediaPage(title = title)
#         except:
#             print(f'Title "{imdb_title}" does not exist on Wikipedia.')

#         for section in section_names:
#             try:
#                 plot = page.section(section)
#             except:
#               print(f'Plot for -{imdb_title}- was not found.')
        
#     try:
#         summary = page.summary
#     except:
#         print(f'Summary for -{imdb_title}- was not found.')        
    
#     wikipedia_title_arr.append(title)
#     movie_plot_arr.append(plot)
#     movie_summary_arr.append(summary)

In [ ]:
# data = pd.read_csv('israel_originated_movies_imdb.csv')
# add_wikipedia()

### Text Preprocessing


### Topic Modeling - Latent Dirichlet Allocation